In [39]:
!pip install pyrubberband

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 671.8/671.8 kB 3.8 MB/s eta 0:00:00
  Created wheel for pyrubberband: filename=pyrubberband-0.3.0-py3-none-any.whl size=4273 sha256=84a14a841f1bb8c4497633d5ed188ed838e383dce76cb2b81823b101aedcc7ca
  Stored in directory: c:\users\ramya\appdata\local\pip\cache\wheels\b8\2d\f0\bb68fbfe67a42c858a79412321d28589218cbfe114c48ce664
Successfully built pyrubberband


In [3]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow import keras
import numpy as np
import pandas as pd
import librosa.display
from tqdm import tqdm
import pickle as pkl

import random
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output, display

import pyrubberband as pyrb
import re
import six
from copy import deepcopy

import keras
import soundfile as sf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from keras.utils import np_utils
from keras.regularizers import l2
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.utils.np_utils import to_categorical

In [4]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, train_metadata, batch_size=32*4):
        'Initialization'
        self.batch_size = batch_size
        self.train_metadata = train_metadata
        self.dim = (36, 5)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor((len(self.train_metadata)*4) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        temp_metadata = []
        for k in range(0, len(indexes), 4):
          temp_metadata.append(self.train_metadata[indexes[k] // 4])

        # Generate data
        X, Y = self.__data_generation(temp_metadata)

        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.train_metadata))

    def __data_generation(self, temp_metadata):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, 1))
        Y = np.empty((self.batch_size), dtype=int)
        idx = 0
        
        # Generate data
        for id, data in enumerate(temp_metadata):
            label = int(data[1])
            sound_clip, sr = librosa.load(data[0])

            original = standard_process(sound_clip)
            
            X[idx,] = original
            Y[idx] = label
            idx += 1

            # time shifted
            rates = [1.5]
            for rate in rates:
              time_stretched = time_stretching(rate, sound_clip)
              X[idx,] = time_stretched
              Y[idx] = label
              idx += 1

            # pitch shifted
            tone_steps = [2]
            for tone_step in tone_steps:
              pitch_shifted = pitch_shifting(tone_step, sound_clip)
              X[idx,] = pitch_shifted
              Y[idx] = label
              idx += 1
            
            # noise added
            noisy_sound = add_noise(sound_clip)
            X[idx,] = noisy_sound
            Y[idx] = label
            idx += 1

        return X, to_categorical(Y, num_classes=10)

In [5]:
def get_model():
  #forming model
  model=Sequential()

  #adding layers and forming the model
  model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(36, 5, 1)))
  model.add(MaxPooling2D(padding="same"))

  model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
  model.add(MaxPooling2D(padding="same"))
  model.add(Dropout(0.3))

  model.add(Flatten())

  model.add(Dense(256,activation="relu"))
  model.add(Dropout(0.4))

  model.add(Dense(512,activation="relu"))
  model.add(Dropout(0.4))

  model.add(Dense(10,activation="softmax"))

  #compiling
  model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

  return model

In [6]:
#extracting data from pkl files
import pickle as pkl

with open("x_data.pickle","rb") as f:
  x_data = pkl.load(f)

with open("y_label.pickle","rb") as f:
  y_label = pkl.load(f)

In [10]:
get_model()